# Laboratorium 2


### Konfiguracja

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.collections as mcoll
import matplotlib.colors as mcolors
from matplotlib.widgets import Button
import json as js


class _Button_callback(object):
    def __init__(self, scenes):
        self.i = 0
        self.scenes = scenes

    def set_axis(self, ax):
        self.ax = ax
        
    def next(self, event):
        self.i = (self.i + 1) % len(self.scenes)
        self.draw()

    def prev(self, event):
        self.i = (self.i - 1) % len(self.scenes)
        self.draw()
        
    def draw(self):
        self.ax.clear()
        for collection in self.scenes[self.i].points:
            if len(collection.points) > 0:
                self.ax.scatter(*zip(*(np.array(collection.points))), c=collection.color, marker=collection.marker)
        for collection in self.scenes[self.i].lines:
            self.ax.add_collection(collection.get_collection())
        self.ax.autoscale()
        plt.draw()

### Interfejsy

[Dostępne kolory](https://matplotlib.org/3.1.1/gallery/color/named_colors.html)

[Dostępne znaczniki punktów](https://matplotlib.org/3.1.1/api/markers_api.html#module-matplotlib.markers)

In [2]:
class Scene:
    def __init__(self, points=[], lines=[]):
        self.points=points
        self.lines=lines

class PointsCollection:
    def __init__(self, points = [], color = None, marker = None):
        self.points = np.array(points)
        self.color = color
        self.marker = marker

class LinesCollection:
    def __init__(self, lines = [], color = None):
        self.color = color
        self.lines = lines
        
    def add(self, line):
        self.lines.append(line)
        
    def get_collection(self):
        if self.color:
            return mcoll.LineCollection(self.lines, colors=mcolors.to_rgba(self.color))
        else:
            return mcoll.LineCollection(self.lines)
            


class Plot:
    def __init__(self, scenes = [], json = None):
        if json is None:
            self.scenes = scenes
        else:
            self.scenes = [Scene([PointsCollection(pointsCol) for pointsCol in scene["points"]], 
                                 [LinesCollection(linesCol) for linesCol in scene["lines"]]) 
                           for scene in js.loads(json)]
        
    def __configure_buttons(self, callback):
        plt.subplots_adjust(bottom=0.2)
        axprev = plt.axes([0.6, 0.05, 0.15, 0.075])
        axnext = plt.axes([0.76, 0.05, 0.15, 0.075])
        bnext = Button(axnext, 'Następny')
        bnext.on_clicked(callback.next)
        bprev = Button(axprev, 'Poprzedni')
        bprev.on_clicked(callback.prev)
        return [bprev, bnext]

    def draw(self):
        plt.close()
        callback = _Button_callback(self.scenes)
        self.widgets = self.__configure_buttons(callback)
        callback.set_axis(plt.axes())
        plt.show()
        callback.draw()
        
    def toJSON(self):
        return js.dumps([{"points": [pointCol.points.tolist() for pointCol in scene.points], 
                          "lines":[linesCol.lines for linesCol in scene.lines]} 
                         for scene in self.scenes])
    

### Przykład użycia

In [3]:
%matplotlib notebook

scenes=[Scene([PointsCollection([(1, 2), (3, 1.5), (2, -1)]), 
               PointsCollection([(5, -2), (2, 2), (-2, -1)], 'green', marker = "^")], 
              [LinesCollection([[(1,2),(2,3)], [(0,1),(1,0)]], 'orange')]), 
        Scene([PointsCollection([(1, 2), (-15, 1.5), (2, -1)], 'red'), 
               PointsCollection([(5, -2), (2, 2), (-2, 1)], 'black')], 
              [LinesCollection([[(-1,2),(-2,3)], [(0,-1),(-1,0)]])])]

plot = Plot(scenes)
plot.draw() 


<IPython.core.display.Javascript object>

# Rozwiązanie

## zad 1) 

In [4]:
%matplotlib notebook
import math
def point_on_cycle(R, t):
    return [R * math.cos((math.pi / 2) * t), R * math.sin((math.pi / 2) * t)]

def point_on_side(side):
    switcher = {
        0: [np.random.uniform(-10, 10), 10],
        1: [np.random.uniform(-10, 10), -10],
        2: [-10, np.random.uniform(-10, 10),],
        3: [10,np.random.uniform(-10, 10)],
    }
    return switcher.get(side)

def rand_linear(n, low, high, pa, pb):
    result = []
    for i in range(n):
        x = np.random.uniform(low, high)
        y = (pa[1]-pb[1])/(pa[0]-pb[0])*x+(pa[1]-(pa[1]-pb[1])/(pa[0]-pb[0])*pa[0])
        result.append([x, y])
    return result

s_a = [0, 0]
s_b = [10, 0]
s_c = [10, 10]
s_d = [0, 10]
diagonal_1 = rand_linear(20, 0, 10, s_a, s_c)
diagonal_2 = rand_linear(20, 0, 10, s_b, s_d)
axis_y = [[0, np.random.uniform(0, 10)] for _ in range(25)]
axis_x = [[np.random.uniform(0, 10), 0] for _ in range(25)]

a = np.random.uniform(-100, 100, (100, 2))
b = np.random.uniform(0, 4, 100)
b = list(map(lambda t: point_on_cycle(10, t), b))
c = list(map(lambda side: point_on_side(side), np.random.randint(0, 4, 100)))
d = [s_a, s_b, s_c, s_d] + diagonal_1 + diagonal_2 + axis_x + axis_y

## zad 2) 

In [5]:
%matplotlib notebook
scenes=[Scene([PointsCollection(a, 'red')]), 
       Scene([PointsCollection(b, 'green')]),
        Scene([PointsCollection(c, 'blue')]),
        Scene([PointsCollection(d, 'purple')]),]

plot = Plot(scenes)
plot.draw()

<IPython.core.display.Javascript object>

## zad 3) 

In [6]:
%matplotlib notebook

def generate_cloud(points_number, low, high):
    cloud = np.random.uniform(low, high, (points_number, 2))
    return cloud


def generate_cloud_collection(points_number, low, high, color):
    cloud = generate_cloud(points_number, low, high)
    return PointsCollection(cloud, color)


def generate_circle(points_number, middle, radius):
    def point_on_cycle(R, middle, t):
        return [R * math.cos((math.pi / 2) * t) + middle[0], R * math.sin((math.pi / 2) * t) + middle[1]]

    parameters_set = np.random.uniform(0, 4, points_number)
    circle = list(map(lambda t: point_on_cycle(radius, middle, t), parameters_set))
    return circle


def generate_circle_collection(points_number, middle, radius, color):
    circle = generate_circle(points_number, middle, radius)
    return PointsCollection(circle, color)


def generate_rectangle(points_number, vertices_list):
    def rand_linear_point(pa, pb):
        if pa[0] == pb[0]:
            return [pa[0], np.random.uniform(pa[1], pb[1])]
        x = np.random.uniform(pa[0], pb[0])
        y = (pa[1]-pb[1])/(pa[0]-pb[0])*x+(pa[1]-(pa[1]-pb[1])/(pa[0]-pb[0])*pa[0])
        return [x, y]
        
    
    def point_on_side(side, vertices_list):
        (p_a, p_b, p_c, p_d) = vertices_list
        switcher = {
            0: rand_linear_point(p_a, p_b),
            1: rand_linear_point(p_b, p_c),
            2: rand_linear_point(p_c, p_d),
            3: rand_linear_point(p_d, p_a),
        }
        return switcher.get(side)
        
    rectangle = list(map(lambda side: point_on_side(side, vertices_list), np.random.randint(0, 4, points_number)))
    return rectangle

def generate_rectangle_collection(points_number, vertices_list, color):
    rectangle = generate_rectangle(points_number, vertices_list)
    return PointsCollection(rectangle, color)


def generate_square(vertices_list, axis_number, diagonal_number):
    (p_a, p_b, p_c, p_d) = vertices_list
    def rand_linear_point(pa, pb):
        if pa[0] == pb[0]:
            return [pa[0], np.random.uniform(pa[1], pb[1])]
        x = np.random.uniform(pa[0], pb[0])
        y = (pa[1]-pb[1])/(pa[0]-pb[0])*x+(pa[1]-(pa[1]-pb[1])/(pa[0]-pb[0])*pa[0])
        return [x, y]
    
    diagonal_1 = [ rand_linear_point(p_a, p_c) for _ in range(diagonal_number)]
    diagonal_2 = [ rand_linear_point(p_b, p_d) for _ in range(diagonal_number)]
    axis_1 = [ rand_linear_point(p_a, p_b) for _ in range(axis_number)]
    axis_2 = [ rand_linear_point(p_a, p_d) for _ in range(axis_number)]
    
    square = diagonal_1 + diagonal_2 + axis_1 + axis_2 + [p_a, p_b, p_c, p_d]
    return square

def generate_square_collection(vertices_list, axis_number, diagonal_number, color):
    square = generate_square(vertices_list, axis_number, diagonal_number)
    return PointsCollection(square, color)

### test if generators work

In [7]:
%matplotlib notebook


scenes=[Scene([generate_cloud_collection(100, -100, 100, 'red')]), 
       Scene([generate_circle_collection(100, [0, 0], 10, 'green')]),
        Scene([generate_rectangle_collection(100, [[-10, -10], [10, -10], [10, 10], [-10, 10]], 'blue')]),
        Scene([generate_square_collection([[0, 0], [10, 0], [10, 10], [0, 10]], 25, 20, 'purple')]),]
plot = Plot(scenes)
plot.draw()

<IPython.core.display.Javascript object>

### Generate modified sets

In [8]:
%matplotlib notebook

a_prim = generate_cloud(50, -250, 100)
b_prim = generate_circle(1000, [0, 5], 13)
c_prim = generate_rectangle(50, [[-13, -7], [12, -5], [14, 11], [-7, 9]])
d_prim = generate_square([[0, 0], [15, 0], [15, 15], [0, 15]], 20, 90)

scenes=[Scene([PointsCollection(a_prim, 'red')]), 
       Scene([PointsCollection(b_prim, 'green')]), 
        Scene([PointsCollection(c_prim, 'blue')]), 
        Scene([PointsCollection(d_prim, 'purple')]),] 
plot = Plot(scenes)
plot.draw()

<IPython.core.display.Javascript object>

## zad 4)

In [9]:
%matplotlib notebook
import time
import copy


def det(a, b, c):
    return a[0] * b[1] * 1 + a[1] * 1 * c[0] + 1 * b[0] * c[1] - 1 * b[1] * c[0] - 1 * c[1] * a[0] - 1 * a[1] * b[0]

def dist(p_a, p_b):
        return math.sqrt((p_a[0] - p_b[0])**2 + (p_a[1] - p_b[1])**2)
    
def angle(point):
    if point[1] == 0:
        return -10000000
    return - point[0] / point[1]

def on_left(a, b ,c):
    e = 10**-6
    return det(a,b,c) > e

def collinear(a,b,c):
    e = 10**-10
    return -e < det(a,b,c) < e

def lines_drawer(points):
    return [[[points[i][0], points[i][1]], [points[i+1][0], points[i+1][1]]] for i in range(len(points)-1)]

def graham(points, debug = False):
    scenes = [Scene([PointsCollection(points, 'red')])]
    print("Starting graham algorithm")
    start = time.time()
    def prepare_points(points):
        prepared_points = copy.deepcopy(points)
        starting_point = prepared_points[0]
        for point in prepared_points:
            if (starting_point[1] > point[1]
                or (starting_point[1] == point[1] and starting_point[0] > point[0])
            ):
                starting_point = point
        (x, y) = starting_point
        for point in prepared_points:
            point[0] -= x
            point[1] -= y
        return prepared_points, x, y
    
    def filter_points(points):
        filtered_points = [points[0]]
        for i in range(1, len(points)-1):
            if abs(angle(points[i]) - angle(points[i+1])) > 10**-6:
                filtered_points.append(points[i])
        filtered_points.append(points[-1])
        return filtered_points
    
    prepared_points, starting_x, starting_y = prepare_points(points)
    if debug: scenes.append(Scene([PointsCollection(prepared_points, 'red')]))
    sorted_points = sorted(prepared_points, key = lambda x: (angle(x), dist(x, [0, 0])))
    filtered_points = filter_points(sorted_points)
    if debug: scenes.append(Scene([PointsCollection(filtered_points, 'red')]))
    
    stack = [filtered_points[0], filtered_points[1], filtered_points[2]]
    i = 3
    while i < len(filtered_points):
        t = len(stack) - 1
        if debug: 
            scenes.append(Scene([PointsCollection(filtered_points, 'red')], 
              [LinesCollection(lines_drawer(stack), 'orange'),
               LinesCollection(lines_drawer([stack[t-1], stack[t], filtered_points[i]]), 'green'),]))
        if t < 1:
            print("error")
            return
        if on_left(stack[t-1], stack[t], filtered_points[i]):
            stack.append(filtered_points[i])
            i = i + 1
        else:
            stack.pop()
    for point in stack:
        point[0] += starting_x
        point[1] += starting_y
    print("Finished graham algorithm")
    print(f'It took {time.time() - start}s')
    if debug: 
        scenes.append(Scene([PointsCollection(points, 'red')], 
              [LinesCollection(lines_drawer(stack + [stack[0]]), 'orange')]))
        return stack, scenes

    return stack


def jarvis(points, debug = False):
    print("Starting jarvis algorithm")
    start = time.time()
    scenes = [Scene([PointsCollection(points, 'red')])]
    point_in_hull_index = 0
    n = len(points)
    for i in range(1, n):
        if (points[point_in_hull_index][1] > points[i][1]
            or (points[point_in_hull_index][1] == points[i][1]
                and points[point_in_hull_index][0] > points[i][0])
        ):
            point_in_hull_index = i
    
    hull = []
    point_in_hull = points[point_in_hull_index]
    starting_point_index = point_in_hull_index
    while True:
        hull.append(point_in_hull)
        next_point_index = (point_in_hull_index + 1) % n 
        next_point = points[next_point_index]
        for i in range(n):
            if debug: scenes.append(Scene([PointsCollection(points, 'red')], 
              [LinesCollection(lines_drawer(hull), 'orange'),
               LinesCollection(lines_drawer([point_in_hull, points[i]]), 'green'),
               LinesCollection(lines_drawer([point_in_hull, next_point]), 'blue')]))
            if on_left(point_in_hull, points[i], next_point)\
                    or (
                collinear(point_in_hull, points[i], next_point) and (
                dist(point_in_hull, points[i]) > dist(point_in_hull, next_point))):
                next_point_index = i
                next_point = points[i]
        point_in_hull_index = next_point_index
        point_in_hull = points[point_in_hull_index]

        if point_in_hull_index == starting_point_index: 
            break

    print("Finished jarvis algorithm")
    print(f'It took {time.time() - start}s')
    if debug:
        scenes.append(Scene([PointsCollection(points, 'red')], 
                  [LinesCollection(lines_drawer(hull + [hull[0]]), 'orange')]))
        return hull, scenes
    return hull

## Testing graham algorithm

### sets from 1)


In [10]:
%matplotlib notebook

scenes=[Scene([PointsCollection(a, 'red'), PointsCollection(graham(a), 'blue')]),
        Scene([PointsCollection(b, 'red'), PointsCollection(graham(b), 'blue')]),
        Scene([PointsCollection(c, 'red'), PointsCollection(graham(c), 'blue')]),
        Scene([PointsCollection(d, 'red'), PointsCollection(graham(d), 'blue')]),
        ]
plot = Plot(scenes)
plot.draw()

Starting graham algorithm
Finished graham algorithm
It took 0.001993894577026367s
Starting graham algorithm
Finished graham algorithm
It took 0.001995086669921875s
Starting graham algorithm
Finished graham algorithm
It took 0.0009989738464355469s
Starting graham algorithm
Finished graham algorithm
It took 0.000997304916381836s


<IPython.core.display.Javascript object>

### sets from 3)

In [11]:
%matplotlib notebook

scenes=[Scene([PointsCollection(a_prim, 'red'), PointsCollection(graham(a_prim), 'blue')]),
        Scene([PointsCollection(b_prim, 'red'), PointsCollection(graham(b_prim), 'blue')]),
        Scene([PointsCollection(c_prim, 'red'), PointsCollection(graham(c_prim), 'blue')]),
        Scene([PointsCollection(d_prim, 'red'), PointsCollection(graham(d_prim), 'blue')]),
        ]
plot = Plot(scenes)
plot.draw()

Starting graham algorithm
Finished graham algorithm
It took 0.000997304916381836s
Starting graham algorithm
error
Starting graham algorithm
Finished graham algorithm
It took 0.0009975433349609375s
Starting graham algorithm
Finished graham algorithm
It took 0.001993894577026367s


<IPython.core.display.Javascript object>

## Testing Jarvis algorithm

### sets from 1)


In [12]:
%matplotlib notebook

scenes=[Scene([PointsCollection(a, 'red'), PointsCollection(jarvis(a), 'blue')]),
        Scene([PointsCollection(b, 'red'), PointsCollection(jarvis(b), 'blue')]),
        Scene([PointsCollection(c, 'red'), PointsCollection(jarvis(c), 'blue')]),
        Scene([PointsCollection(d, 'red'), PointsCollection(jarvis(d), 'blue')]),
        ]
plot = Plot(scenes)
plot.draw()

Starting jarvis algorithm
Finished jarvis algorithm
It took 0.01396489143371582s
Starting jarvis algorithm
Finished jarvis algorithm
It took 0.02293992042541504s
Starting jarvis algorithm
Finished jarvis algorithm
It took 0.002991914749145508s
Starting jarvis algorithm
Finished jarvis algorithm
It took 0.0019948482513427734s


<IPython.core.display.Javascript object>

### sets from 3)

In [13]:
%matplotlib notebook

scenes=[Scene([PointsCollection(a_prim, 'red'), PointsCollection(jarvis(a_prim), 'blue')]),
        Scene([PointsCollection(b_prim, 'red'), PointsCollection(jarvis(b_prim), 'blue')]),
        Scene([PointsCollection(c_prim, 'red'), PointsCollection(jarvis(c_prim), 'blue')]),
        Scene([PointsCollection(d_prim, 'red'), PointsCollection(jarvis(d_prim), 'blue')]),
        ]
plot = Plot(scenes)
plot.draw()

Starting jarvis algorithm
Finished jarvis algorithm
It took 0.00797891616821289s
Starting jarvis algorithm
Finished jarvis algorithm
It took 2.010667085647583s
Starting jarvis algorithm
Finished jarvis algorithm
It took 0.0009958744049072266s
Starting jarvis algorithm
Finished jarvis algorithm
It took 0.0019953250885009766s


<IPython.core.display.Javascript object>

## Saving results to file

In [14]:
%matplotlib notebook

def save_to_file(results, file_name):
    with open(file_name, 'w') as outfile:
        js.dump(results, outfile)
    
save_to_file(jarvis(d_prim), "plot1.json")
 

Starting jarvis algorithm
Finished jarvis algorithm
It took 0.0029931068420410156s


## Visualising the algorithms

### Graham

In [15]:
%matplotlib notebook   

results, scenes = graham(a_prim, debug = True)

plot = Plot(scenes)
plot.draw()

Starting graham algorithm
Finished graham algorithm
It took 0.03191399574279785s


<IPython.core.display.Javascript object>

### Jarvis

In [16]:
%matplotlib notebook   

results, scenes = jarvis(a_prim, debug = True)

plot = Plot(scenes)
plot.draw()

Starting jarvis algorithm
Finished jarvis algorithm
It took 0.015957355499267578s


<IPython.core.display.Javascript object>

## Jarvis vs Graham

In [17]:
big_cloud = generate_cloud(100000, -250, 100)
big_circle = generate_circle(1000, [0, 0], 100)
big_rectangle = generate_rectangle(100000, [[-1000, -1000], [1000, -1000], [1000, 1000], [-1000, 1000]])
big_square = generate_square([[0, 0], [15, 0], [15, 15], [0, 15]], 10000, 10000)

print("random points")
jarvis(big_cloud)
graham(big_cloud)
print("random circle")
jarvis(big_circle)
graham(big_circle)
print("random rectangle")
jarvis(big_rectangle)
graham(big_rectangle)
print("random square")
jarvis(big_square)
graham(big_square)

random points
